In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import json
import glob
from numpyencoder import NumpyEncoder
import re

In [2]:
PATH="Dataset"
METADATA= glob.glob(f"{PATH}/**/**.json")

In [4]:
def save_meta_data(file_name,dict_file):
    with open(f'{file_name}.json', 'w') as fp:
        json.dump(dict_file, fp,  indent=4,cls=NumpyEncoder)

def read_meta_data (path):
    with open(path, 'r') as j:
        contents = json.loads(j.read())
        return contents
        
def del_data(files_name,folder_name="Dataset"):
    try:
        for d in files_name:
            os.remove(f"{folder_name}/{d}")
    except:
        os.remove(f"{folder_name}/{files_name}")

In [4]:
buff = glob.glob(f"{PATH}/T_316000_233500_NE_T_316000_233500_SW/**.txt")
data = np.array([i.split("/") for i in buff if "building" in i or "buidling" in i])
new= data[:,-1].copy()
new = [i.split("_") for i in new]
for i,_ in enumerate(new):
    if len(new[i])==2:
        new[i][1] = new[i][1].split(".")
        new[i][1][0]= new[i][1][0].zfill(3)
        new[i][1]= ".".join(new[i][1])
    else:
        new[i][1]=new[i][1].zfill(3)
    new[i]= "_".join(new[i])
index = np.where(data[:,-1]!=new)[0]
if index.size != 0:
    old_data= data[:,-1]
    METADATA_buff = read_meta_data(f'{PATH}/T_316000_233500_NE_T_316000_233500_SW/meta_data.json')
    OLD_METADATA_buff = METADATA_buff.copy()
    for i in index:
        try:
            data_edited= METADATA_buff.pop(old_data[i][:-4])
        except:
            continue
        list_new_data=new[i].split("/")
        METADATA_buff[list_new_data[-1][:-4]]={
                                                "path":f"T_316000_233500_NE_T_316000_233500_SW/{new[i]}",
                                                "jumlah_point":data_edited["jumlah_point"]
                                                }

    print(index)
    save_meta_data(f'{PATH}/T_316000_233500_NE_T_316000_233500_SW/meta_data',METADATA_buff)
    for idx in index:
        os.rename(f"{PATH}/T_316000_233500_NE_T_316000_233500_SW/{old_data[idx]}",f"{PATH}/T_316000_233500_NE_T_316000_233500_SW/{new[idx]}")
    #     # print()
    #     # print(old_data)
METADATA= glob.glob(f"{PATH}/**/**.json")

In [5]:
def get_list_per_instace(str_name ,file_list,Area,dict_file):
    result_dict = {}
    id_ns = np.unique(np.array([b.split("_")[1] for b in dict_file],dtype="int16"))
    zf = 3 if Area ==  "T_316000_233500_NE_T_316000_233500_SW" or Area=="T_315500_234500_SE" else 2
    for idn in id_ns :
        r = re.compile(f"{str_name}_{str(idn).zfill(zf)}")
        result_dict[f"{str_name}_{str(idn).zfill(zf)}"] = list(filter(r.match, file_list)) 
    return result_dict

for i,_ in tqdm.tqdm(enumerate(METADATA),total=len(METADATA)):
    Area = METADATA[i].split("/")[1]
    md = read_meta_data(METADATA[i])
    file_names = md.keys()
    building,ground,undefined,vegetation=[],[],[],[]
    label = set([i.split("_")[0] for i in file_names])
    for fn in file_names:
        if 'building' in fn.lower() or "buidling" in fn.lower():
            building.append(fn)
        if 'ground' in fn.lower():
            ground.append(fn)
        if 'undefined' in fn.lower() or "undifined" in fn.lower() or "undefined" in fn.lower() or 'Undefined' in fn.lower():
            undefined.append(fn)
        if 'vegetation'in fn.lower():
            vegetation.append(fn)
    dict_area={
        "building":get_list_per_instace(str_name = "building",file_list = building,dict_file=building,Area=Area),
        "vegetation":vegetation,
        "ground":ground,
        "undefined":undefined
        }
    save_meta_data(f"map_ins_{Area}",dict_area)

def maping_meta_data(class_name,map_data,meta_data):
    inst_id = {}
    try:
        for count,key in enumerate(map_data[class_name]):
            try:
                k_split = key.split("_")
                if len(k_split)==3:
                    _,sub_class,key_id = k_split
                else :
                    sub_class,key_id[:-4] = k_split
            except:
                sub_class,key_id=key,0
                
            inst_id[str(count).zfill(3)]={
                "total_point": meta_data[key]["jumlah_point"],
                "path": meta_data[key]["path"],
                "sub_class": sub_class,
                "id": str(int(key_id)-1).zfill(3)
                }
    except Exception as e:
        print(class_name)
        print(meta_data[class_name])
        inst_id["000"]={
            "total_point": meta_data[class_name]["jumlah_point"],
            "path": meta_data[class_name]["path"],
            "sub_class": "000",
            "id": "000"
            }
    return inst_id
    
def map_to_inst(map_data_json,meta_data_json):
    map_data = read_meta_data(map_data_json)
    meta_data= read_meta_data(meta_data_json)
    for k in map_data.keys():
        if k == "building":
            inst_id_building = {}
            list_build_id= list(map_data[k].keys())
            for id_b in list_build_id:
                id_ins = str(int(id_b.split("_")[-1])-1).zfill(3)
                total_point = sum([meta_data[i]["jumlah_point"] for i in map_data[list(map_data.keys())[0]][id_b]])
                list_all_txt = [meta_data[i]["path"] for i in map_data[list(map_data.keys())[0]][id_b]]
                inst_id_building[id_ins]={
                    "total_point":total_point,
                    "path":list_all_txt
                    }
        elif k=="vegetation" :
            inst_id_vegetation = maping_meta_data("vegetation",map_data,meta_data)
        elif k=="ground" :
            inst_id_ground = maping_meta_data("ground",map_data,meta_data)
        else :
            inst_id_undefined = maping_meta_data("undefined",map_data,meta_data)
    return [inst_id_building,inst_id_vegetation,inst_id_ground,inst_id_undefined]

for i,_ in enumerate(METADATA):
    area=METADATA[i].split("/")[1]
    ins = map_to_inst(f"map_ins_{area}.json",METADATA[i])
    save_meta_data(f"map_ins_{area}",{"building":ins[0],"vegetation":ins[1],"ground":ins[2],'undefined':ins[3]})

100%|██████████| 13/13 [00:00<00:00, 14.86it/s]


In [10]:
MAPTOINST_META_LIST= glob.glob("map_**")
def get_combining_building_data(map_data_json = MAPTOINST_META_LIST,buff_folder="Dataset") :   
    for i,meta_name in tqdm.tqdm(enumerate(map_data_json),total=13):
        map_data = read_meta_data(map_data_json[i])["building"]
        buff_folder = f"all_data_building/{map_data_json[i][8:-5]}"
        os.makedirs(buff_folder,exist_ok=True)
        map_data_id= list(map_data.keys())
        for mdi in map_data_id:
            list_data_txt = map_data[mdi]["path"]
            # Start from init = 0 and append until last
            all_data = np.loadtxt(f"Dataset/{list_data_txt[0]}")[:,:6]
            for idx in range(1,len(list_data_txt)):
                try:
                    buff = np.loadtxt(f"Dataset/{list_data_txt[idx]}")[:,:6]
                    all_data = np.append(all_data,buff,axis=0)
                except :
                    buff = np.loadtxt(f"Dataset/{list_data_txt[idx]}")[:6]
                    all_data = np.append(all_data,[buff],axis=0)
            del_data(list_data_txt)
            if "T_316000_233500_NE_T_316000_233500_SW" in meta_name:
                data_building_index = np.where(all_data[:,0]<318000)
                data_bulding_sub_building_index = np.where(all_data[:,0]>318000)
                data_building = all_data[data_building_index]
                data_bulding_sub_building = all_data[data_bulding_sub_building_index]
                data_bulding_sub_building[:,0] = data_bulding_sub_building[:,0]-2000
                data_bulding_sub_building[:,1] = data_bulding_sub_building[:,1]-3619
                data_bulding_sub_building[:,2] = data_bulding_sub_building[:,2]+40
                all_data = np.append(data_building,data_bulding_sub_building,axis=0)
            save_npy_loc = f"{buff_folder}/building_{mdi}.npy"
            map_data[mdi]["path"] = save_npy_loc
            np.save(save_npy_loc,all_data,allow_pickle=True)
        old_map = read_meta_data(map_data_json[i])
        old_map["building"]= map_data
        save_meta_data(f"{map_data_json[i][8:-5]}",old_map)
        os.remove(f"{map_data_json[i]}")
get_combining_building_data()

 92%|█████████▏| 12/13 [42:24<03:32, 212.08s/it] 


In [12]:
get_combining_building_data(map_data_json=["map_ins_T_316000_234000_SW.json"])

  0%|          | 0/13 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'map_ins_T_316000_234000_SW.json'

In [11]:
MAPTOINST_META_LIST

['map_ins_T_316000_233500_NE_T_316000_233500_SW.json',
 'map_ins_T_316000_233500_NW.json',
 'map_ins_T_316000_233500_SE.json',
 'map_ins_T_316000_234000_NE.json',
 'map_ins_T_316000_234000_NW.json',
 'map_ins_T_316000_234000_SE.json',
 'map_ins_T_316000_234000_SW.json',
 'map_ins_T_316500_234000_SW_T_316500_233500_NW.json',
 'map_ins_T_315500_234500_SE.json',
 'map_ins_T_315500_233500_NE_T_315500_234000_SE.json',
 'map_ins_T_315500_234500_NE.json',
 'map_ins_T_315500_234500_NW.json']